In [275]:
# Import 
import pandas as pd
import re
import os 
from pathlib import Path

In [278]:
# Get RAW directory from notebook and snakemake .. 
RAW_DIRECTORY = os.getcwd().replace("parsers","") + "/raw"

In [279]:
RAW_FILE = f"{RAW_DIRECTORY}/genes_to_phenotype.txt"
GENE_MAP =  f"{RAW_DIRECTORY}/gene_map.csv"

In [280]:
# Create a genemap :   entrez_id : ensembl_id 
genemap = pd.read_csv(GENE_MAP, sep="\t")
genemap = genemap[["entrez_id","ensembl_gene_id"]].drop_duplicates().set_index("entrez_id").to_dict()["ensembl_gene_id"]

In [273]:
headers = next(open(RAW_FILE)).replace("#Format:","").strip().split("<tab>")
df = pd.read_csv(RAW_FILE, 
            comment="#", 
            delimiter="\t",
            names=["gene_id","gene","hpo","term_name","raw","freq","additional","source","omim"]
           )

# Join HPO terms by gene 
gdf = df.groupby("gene_id").agg({"hpo": lambda x: "|".join(x)}).reset_index().copy()


gdf["ensembl_id"] = gdf["gene_id"].map(genemap)

# debug >>>>
skip_genes = df[df["gene_id"].isin(gdf[gdf["ensembl_id"].isna()]["gene_id"].to_list())]["gene"].unique()
# debug <<<<

gdf = gdf.dropna()[["ensembl_id","hpo"]]

gdf

,ensembl_id,hpo
0,ENSG00000090861,HP:0003477|HP:0001761|HP:0001765|HP:0002460|HP...
1,ENSG00000183044,HP:0006829|HP:0001250|HP:0001321|HP:0025430|HP...
2,ENSG00000165029,HP:0001712|HP:0001744|HP:0003693|HP:0002240|HP...
3,ENSG00000107331,HP:0002066|HP:0001260|HP:0003141|HP:0001250|HP...
4,ENSG00000167972,HP:0002615|HP:0012418|HP:0000961|HP:0001662|HP...
...,...,...
4699,ENSG00000225484,HP:0002460|HP:0001337|HP:0000508|HP:0003581|HP...
4700,ENSG00000281344,HP:0003641|HP:0100598|HP:0008071|HP:0011419|HP...
4701,ENSG00000244040,HP:0001701|HP:0002637|HP:0001824|HP:0100654|HP...
4703,ENSG00000284395,HP:0002014|HP:0011473|HP:0000007


In [241]:
# UNIQUEMENT POUR SNAKEMAKE .... 
#output = "/tmp/test.csv"

output = snakemake.output[0]

descriptions = (
("title", "hpo"),
("description", "HPO terms"),
("version", "no idea"),
("ensembl_id", "ensembl gene id "),
("hpo", "HPO annotation list ")
)

with open(output,"w") as f:
    for d in descriptions:
        f.write("#" + "\t".join(d) +"\n")
        
    gdf.to_csv(f,sep="\t", index=False)

NameError: name 'snakemake' is not defined

In [240]:
!head /tmp/test.csv

#title	hpo
#description	HPO terms
#version	no idea
#ensembl_id	ensembl gene id 
#hpo	HPO annotation list 
ensembl_id	hpo
ENSG00000090861	HP:0003477|HP:0001761|HP:0001765|HP:0002460|HP:0001284|HP:0003828|HP:0009027|HP:0000407|HP:0003431|HP:0000006|HP:0002936|HP:0003202|HP:0002059|HP:0001263|HP:0001511|HP:0002827|HP:0002072|HP:0000252|HP:0004322|HP:0009830|HP:0000643|HP:0200134|HP:0001508|HP:0003577|HP:0000639|HP:0001284|HP:0001257|HP:0001250|HP:0001290|HP:0003429|HP:0001298|HP:0000007|HP:0002063|HP:0000508|HP:0001268|HP:0001336|HP:0001290|HP:0001337|HP:0001298|HP:0000648|HP:0001263|HP:0002521|HP:0001265|HP:0002020|HP:0000252|HP:0100710|HP:0001249|HP:0002421|HP:0001273|HP:0011968|HP:0002133|HP:0000494|HP:0007018|HP:0002509|HP:0002376|HP:0100660|HP:0012447|HP:0004322|HP:0000504|HP:0001508|HP:0000546|HP:0002317|HP:0001251|HP:0001257|HP:0000717|HP:0002059|HP:0000348|HP:0000750|HP:0000639|HP:0001558|HP:0010844|HP:0000668|HP:0002355
ENSG00000183044	HP:0006829|HP:0001250|HP:0001321|HP:0025430|